In [14]:
import geopandas as gpd
import fiona
import folium

In [5]:
# Path to the geodatabase
gdb_path = "pmbc_parcel_fabric_poly_svw.gdb"

In [6]:
# List all available layers in the geodatabase
layers = fiona.listlayers(gdb_path)
print("Available layers:", layers)

# Load a specific layer
layer_name = layers[0]  # Choose the layer you want to load
gdf = gpd.read_file(gdb_path, layer=layer_name)

# Display the GeoDataFrame
print(gdf.head())

Available layers: ['pmbc_parcel_fabric_poly_svw']


/home/liamgraham123/landwise_analytica/landwise_env/lib/python3.11/site-packages/pyogrio/raw.py:198: RuntimeWarning: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined
  return ogr_read(


   PARCEL_FABRIC_POLY_ID PARCEL_NAME PLAN_NUMBER  PIN         PID  \
0                 719065        Road    EPP39710  NaN         NaN   
1                 719070   012462314     NWP2004  NaN  12462314.0   
2                 719075   017150728    NWP88218  NaN  17150728.0   
3                 719080   013190121     NO_PLAN  NaN  13190121.0   
4                 719085   023452145    LMP28743  NaN  23452145.0   

  PARCEL_STATUS PARCEL_CLASS        OWNER_TYPE PARCEL_START_DATE  \
0        Active         Road  Local Government               NaT   
1        Active  Subdivision  Local Government        1997-02-13   
2        Active  Subdivision  Local Government               NaT   
3        Active  Subdivision  Local Government               NaT   
4        Active  Subdivision  Local Government        1996-06-25   

      MUNICIPALITY                  REGIONAL_DISTRICT        WHEN_UPDATED  \
0  Surrey, City of  Metro Vancouver Regional District 2024-08-18 17:20:23   
1  Surrey, City of  Me

In [7]:
print(gdf.info())


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2428660 entries, 0 to 2428659
Data columns (total 16 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   PARCEL_FABRIC_POLY_ID  int32         
 1   PARCEL_NAME            object        
 2   PLAN_NUMBER            object        
 3   PIN                    float64       
 4   PID                    float64       
 5   PARCEL_STATUS          object        
 6   PARCEL_CLASS           object        
 7   OWNER_TYPE             object        
 8   PARCEL_START_DATE      datetime64[ms]
 9   MUNICIPALITY           object        
 10  REGIONAL_DISTRICT      object        
 11  WHEN_UPDATED           datetime64[ms]
 12  FEATURE_AREA_SQM       float64       
 13  FEATURE_LENGTH_M       float64       
 14  SE_ANNO_CAD_DATA       object        
 15  geometry               geometry      
dtypes: datetime64[ms](2), float64(4), geometry(1), int32(1), object(8)
memory usage: 287.2+ MB
None


In [8]:
print(gdf.columns)


Index(['PARCEL_FABRIC_POLY_ID', 'PARCEL_NAME', 'PLAN_NUMBER', 'PIN', 'PID',
       'PARCEL_STATUS', 'PARCEL_CLASS', 'OWNER_TYPE', 'PARCEL_START_DATE',
       'MUNICIPALITY', 'REGIONAL_DISTRICT', 'WHEN_UPDATED', 'FEATURE_AREA_SQM',
       'FEATURE_LENGTH_M', 'SE_ANNO_CAD_DATA', 'geometry'],
      dtype='object')


In [9]:
print(gdf.describe())


       PARCEL_FABRIC_POLY_ID           PIN           PID  \
count           2.428660e+06  2.213740e+05  2.238820e+06   
mean            5.717586e+06  2.713667e+07  1.588954e+07   
min             7.189600e+05  0.000000e+00  1.900000e+01   
25%             3.216968e+06  6.960948e+06  7.004229e+06   
50%             5.727300e+06  1.612452e+07  1.457528e+07   
75%             8.212730e+06  2.973129e+07  2.609435e+07   
max             1.070547e+07  9.401034e+08  3.233771e+07   
std             2.885490e+06  3.005863e+07  1.006753e+07   

                PARCEL_START_DATE                WHEN_UPDATED  \
count                     1470858                     2428660   
mean   2007-02-07 22:50:14.764000  2021-01-20 06:38:32.524000   
min           1848-07-17 00:00:00         2016-04-21 02:53:19   
25%           2003-04-30 00:00:00         2019-01-12 14:03:51   
50%           2010-12-22 00:00:00         2020-01-24 15:13:34   
75%           2016-04-05 00:00:00         2023-05-04 14:34:33   
max 

In [10]:
for index, row in gdf.iterrows():
    print(f"Index: {index}")
    print(f"Geometry: {row['geometry']}")  # The geometry column
    print(f"Attributes: {row}")  # Access all attributes for that row
    print()  # Add spacing between rows for clarity

    if index > 2:
        break

Index: 0
Geometry: MULTIPOLYGON (((1231687.5502000004 468848.76779999956, 1231636.3427000009 468846.3457999993, 1231636.1515999995 468851.21990000084, 1231637.8500999995 468850.8663999997, 1231639.4945999999 468850.3136, 1231641.0617999993 468849.5692999996, 1231687.4104999993 468851.76139999926, 1231687.5502000004 468848.76779999956)))
Attributes: PARCEL_FABRIC_POLY_ID                                               719065
PARCEL_NAME                                                           Road
PLAN_NUMBER                                                       EPP39710
PIN                                                                    NaN
PID                                                                    NaN
PARCEL_STATUS                                                       Active
PARCEL_CLASS                                                          Road
OWNER_TYPE                                                Local Government
PARCEL_START_DATE                                

In [12]:
for g, geometry in enumerate(gdf['geometry']):
    print(g, geometry)
    if g > 2:
        break

0 MULTIPOLYGON (((1231687.5502000004 468848.76779999956, 1231636.3427000009 468846.3457999993, 1231636.1515999995 468851.21990000084, 1231637.8500999995 468850.8663999997, 1231639.4945999999 468850.3136, 1231641.0617999993 468849.5692999996, 1231687.4104999993 468851.76139999926, 1231687.5502000004 468848.76779999956)))
1 MULTIPOLYGON (((1235402.8304999992 468619.9799000006, 1235383.4484 468619.20380000025, 1235382.0292000007 468653.22289999947, 1235401.4209000003 468654.00909999944, 1235402.8304999992 468619.9799000006)))
2 MULTIPOLYGON (((1240199.7346 460082.2667999994, 1240153.9411999993 460080.4321999997, 1240147.9464999996 460080.1963999998, 1240151.329500001 459997.3770000003, 1240161.1462999992 459981.8032000009, 1240143.1318999995 459981.1043999996, 1240125.1285999995 459980.38599999994, 1240107.1258000005 459979.6576000005, 1240092.0076000001 459979.05450000055, 1240089.1223000009 459978.93930000067, 1240071.1191000007 459978.22110000066, 1240068.6170000006 459978.1242999993, 

In [13]:
# gdf.plot() #takes a long time



KeyboardInterrupt

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f89c711ae90>>
Traceback (most recent call last):
  File "/home/liamgraham123/landwise_analytica/landwise_env/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


Error in callback <function flush_figures at 0x7f88e6ddf060> (for post_execute), with arguments args (),kwargs {}:



KeyboardInterrupt



In [15]:
#Takes a long time

center = gdf.geometry.centroid[0].coords[0][::-1]  # Get lat, lon from the first geometry's centroid

# Create a folium map centered at the data location
m = folium.Map(location=center, zoom_start=12)

# Add GeoDataFrame to the folium map
folium.GeoJson(gdf).add_to(m)

# Display the map in a Jupyter notebook (or save it to HTML)
m.save("map.html")

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f89c711ae90>>
Traceback (most recent call last):
  File "/home/liamgraham123/landwise_analytica/landwise_env/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


KeyboardInterrupt: 

In [21]:
center = gdf.geometry.centroid[1].coords[0][::-1]  # Get lat, lon from the first geometry's centroid


In [22]:
center

(468636.60540651286, 1235392.432649626)

In [20]:
gdf.PID

0                 NaN
1          12462314.0
2          17150728.0
3          13190121.0
4          23452145.0
              ...    
2428655    18872344.0
2428656    18872883.0
2428657    18872387.0
2428658    18872875.0
2428659    18872417.0
Name: PID, Length: 2428660, dtype: float64